# Corona Daily Death Report

#### Submitted by
###### Balaji Mohan
###### balajimohan28@gmail.com
###### 040 349 2407

#### 1. Python: Download a timeseries of daily deaths per country
Datasets are downloaded and loaded into the corresponding work directory folder.

#### 2. Python: Convert the table so that each country and each day is a separate row

In [208]:
import pandas as pd
import sqlite3
deaths_total_csv = pd.read_csv("time_series_covid19_deaths_global.csv")
deaths_total_df = pd.DataFrame()
deaths_total_df = deaths_total_csv.melt(id_vars=["Province/State", "Country/Region", "Lat", "Long"], 
        var_name="Date_value", 
        value_name="Death_Value")
deaths_total_df['Date_value'] = pd.to_datetime(deaths_total_df['Date_value'])
deaths_total_df.rename(columns = {'Province/State':'Province_State', 'Country/Region':'Country_Region', 'Long':'Longitude'}, inplace = True) 
deaths_total_df = deaths_total_df.sort_values(by=["Country_Region", "Date_value"])
deaths_total_df.reset_index(drop=True)
deaths_total_df['id'] = range(1, len(deaths_total_df) + 1)
#deaths_total_df.head(2)

#### 3. Python: Provide code to upload the table from step 3 into an SQL table named deaths_total

In [209]:
conn = sqlite3.connect('Corona.db')
c = conn.cursor()
c.execute('CREATE TABLE deaths_total (Province_State text, Country_Region text, Lat float, Longitude float, Date_value Date, Death_Value int, id int)')
conn.commit()
deaths_total_df.to_sql('deaths_total', conn, if_exists='replace', index = False)
 
deaths_total = c.execute('''  
SELECT * FROM deaths_total
''')

print(type(deaths_total))
# for row in c.execute('SELECT * FROM deaths_total'):
#     print(row)


<class 'sqlite3.Cursor'>


In [207]:
#c.execute('DROP TABLE deaths_total')

#### 4. Python: From the data in step 2, calculate the daily change in deaths for each country

In [210]:
deaths_change = pd.DataFrame()
deaths_change = deaths_total_df

country = deaths_change['Country_Region'].tolist()
value = deaths_change['Death_Value'].tolist()
daily_change_value = []
flag = 0
for i in range(len(country)):
    if i+1 < len(country): 
        if country[i] == country[i+1]:
            if flag == 0:
                current_value = value[i]
            else:
                current_value = value[i] - value[i-1]
            flag = 1
        else:
            flag = 0
            current_value = value[i] - value[i-1]
        daily_change_value.append(current_value)   
    else:
        current_value = value[i] - value[i-1]
        daily_change_value.append(current_value)
        
deaths_change['daily_change_value'] = daily_change_value

#deaths_change.to_csv("Final.csv")
#print(deaths_change)

#### 5. Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

In [212]:
# conn = sqlite3.connect('Corona.db')
# c = conn.cursor()
c.execute('CREATE TABLE deaths_change_python (Province_State text, Country_Region text, Lat float, Longitude float, Date_value Date, Death_Value int, daily_change_value int, id int)')
conn.commit()
deaths_change.to_sql('deaths_change_python', conn, if_exists='replace', index = False)
 
deaths_change_python = c.execute('''  
SELECT * FROM deaths_change_python
''')

print(type(deaths_change_python))
# for row in c.execute('SELECT * FROM deaths_change_python'):
#     print(row)


<class 'sqlite3.Cursor'>


In [211]:
#c.execute('DROP TABLE deaths_change_python')

#### 6. SQL: Provide SQL code to calculate the daily change for each country using only the data from deaths_total and save it into an SQL table named deaths_change_sql

In [214]:
# conn = sqlite3.connect('Corona.db')
# c = conn.cursor()
c.execute('CREATE TABLE deaths_change_sql (Province_State text, Country_Region text, Lat float, Longitude float, Date_value Date, Death_Value int, daily_change_value int, id int)')
conn.commit()
deaths_change.to_sql('deaths_change_sql', conn, if_exists='replace', index = False)
deaths_change_sql = c.execute('''  
SELECT a.id, a.Country_Region, a.Province_State, a.Lat, a.Longitude, a.Date_value, a.Death_Value,
 CASE
   WHEN a.id = 1 THEN a.Death_Value
   ELSE (a.Death_Value - IFNULL((SELECT sum(b.Death_Value) from deaths_total b
          WHERE b.id = a.id - 1
          AND b.Country_Region = a.Country_Region
         -- AND b.Date_value = b.Date_value
          AND b.Date_value between b.Date_value and a.Date_value
          GROUP BY b.Country_Region,b.Date_value),0) ) END as daily_change_val
FROM deaths_total a
WHERE a.Country_Region = a.Country_Region;
''')

print(type(deaths_change_sql))
# for row in c.execute('SELECT * FROM deaths_change_sql'):
#     print(row)


<class 'sqlite3.Cursor'>


In [213]:
#c.execute('DROP TABLE deaths_change_sql')

OperationalError: no such table: deaths_change_sql